In [ ]:
# allocation Coding 1_20200517
배정 Coding
○환산피순으로 근거리 고객 한명씩 배정
 1.엑셀자료 불러오기
 2.FC별 환산피순으로 근거리 고객 배정
  - 근거리 : 자표거리 계산

In [1]:
# 0.환경설정
# 0.1라이브러리 불러오기
import numpy as np
import pandas as pd
import os
import numbers
import math

In [2]:
# 1.엑셀자료 불러오기
base_dir = 'C:/Users/xnote/Documents/py'
excel_file = 'allocation Coding 1.xlsx'
excel_dir = os.path.join(base_dir, excel_file)

In [3]:
# FC
em01 = pd.read_excel(excel_dir, # write your directory here
                              sheet_name = 'em', 
                              header = 0, 
                              #names = ['region', 'sales_representative', 'sales_amount'], 
#                               dtype = {'region': str, 
#                                          'sales_representative': np.int64, 
#                                          'sales_amount': float}, # dictionary type
#                               index_col = 'em_num',
                              na_values = 'NaN'
#                               thousands = ',', 
#                               nrows = 10, 
                              )
em01

,em_num,em_name,money,latitude,hardness
0,1,A,41,126.861717,37.874623
1,2,B,48,126.569316,37.122076
2,3,D,5,126.806583,37.531962
3,4,E,30,126.058275,37.535272
4,5,F,21,126.775502,37.807801
5,6,G,19,126.666107,37.427021
6,7,H,17,126.244399,37.906478
7,8,J,15,126.027184,37.185274
8,9,I,15,126.224037,37.465374
9,10,K,99,126.867068,37.205859


In [4]:
# 고객
cu01 = pd.read_excel(excel_dir, # write your directory here
                              sheet_name = 'cu', 
                              header = 0, 
                              #names = ['region', 'sales_representative', 'sales_amount'], 
#                               dtype = {'region': str, 
#                                          'sales_representative': np.int64, 
#                                          'sales_amount': float}, # dictionary type
#                               index_col = 'cu_num',
                              na_values = 'NaN'
#                               thousands = ',', 
#                               nrows = 10, 
                              )
print(cu01)

    cu_num cu_name    latitude   hardness
0        1       A  126.744060  37.380985
1        2       B  126.228165  37.208757
2        3       D  126.797383  37.293906
3        4       E  126.567757  37.937422
4        5       F  126.840459  37.341861
5        6       G  126.701225  37.062907
6        7       H  126.183110  37.059070
7        8       J  126.423235  37.208687
8        9       I  126.964673  37.674086
9       10       K  126.652575  37.954124
10      11       C  126.885017  37.204106
11      12       L  126.788368  37.265269
12      13       M  126.582492  37.545942
13      14       N  126.411957  37.718648
14      15       O  126.410547  37.233936
15      16       P  126.966812  37.040045
16      17       Q  126.449354  37.050561
17      18       R  126.243210  37.200198
18      19       S  126.401271  37.551730
19      20       T  126.580149  37.992907


In [5]:
#  2.FC별 환산피순으로 근거리 고객 배정
#   - 근거리 : 자표거리 계산

# 2.1  거리함수 선정
class GeoUtil:
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi/180)
    
    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        R = 6371 # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2-x1)    
        dLat = GeoUtil.degree2radius(y2-y1)

        a = math.sin(dLat/2) * math.sin(dLat/2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
              *math.cos(GeoUtil.degree2radius(y2)) \
              *math.sin(dLon/2) * math.sin(dLon/2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return round(R * b, round_decimal_digits)

    @staticmethod
    def get_euclidean_distance(x1, y1, x2, y2, round_decimal_digits=5):        
        """
        유클리안 Formula 이용하여 (x1,y1)과 (x2,y2) 점의 거리를 반환
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        dLon = abs(x2-x1) # 경도 차이
        if dLon >= 180:   # 반대편으로 갈 수 있는 경우
            dLon -= 360   # 반대편 각을 구한다
        dLat = y2-y1      # 위도 차이
        return round(math.sqrt(pow(dLon,2)+pow(dLat,2)),round_decimal_digits)

In [96]:
#  2.FC별 환산피순으로 근거리 고객 배정
#   - 근거리 : 자표거리 계산

# 2.2 FC별 환산피순 정렬
em01.sort_values(by=['money'], axis=0, ascending = False, inplace = True)

# 2.3 FC별 최소거리 고객 배정
## 데이터 프레임 길이구해서 for range 값에 추가 -> 결과를 FC 데이터에 결합해서 배정 데이터 생성
i = 0; j = 0;
len1 = (len(em01)); len2 = (len(cu01));
l1 = []; dis1 = []; mindis1 = [];
all01 = pd.DataFrame({'em_num':[], 'cu_num':[], 'distance':[] })

for i in range(0, len1):
    em_list1 = list(em01.loc[i])
    dis1 = []
#     print("i", i)
#     print(em_list1)
    for j in range(0, len2):
#         print("j", j)
        dis1.append(GeoUtil.get_harversion_distance(em_list1[3], em_list1[4], cu01.loc[j,"latitude"], cu01.loc[j,"hardness"])) #  거리계산
    
#     all01 = pd.concat([all01, pd.DataFrame({'em_num':em_list1[0], 'cu_num':cu01.loc[dis1.index(max(dis1)-1),"cu_num"]])
    print(dis1)
    print(min(dis1))

    all01 = pd.concat([ all01, pd.DataFrame({'em_num':em_list1[0], 'cu_num':[cu01.loc[dis1.index(min(dis1)),"cu_num"]] 
                     , 'distance':min(dis1) }) ]
                     , ignore_index = True)
print(all01)

[55.85948, 92.74777, 64.82117, 26.71918, 59.27001, 91.36329, 108.6766, 83.53296, 24.06482, 20.36569, 74.58642, 68.06471, 44.03472, 43.15659, 81.59082, 93.26331, 98.59468, 92.72484, 54.12553, 27.97834]
20.36569
[32.68079, 31.72897, 27.80303, 90.66241, 34.25634, 13.42284, 34.96494, 16.13401, 70.62074, 92.81026, 29.42475, 25.09981, 47.14615, 67.77564, 18.77684, 36.42223, 13.28416, 30.17549, 50.03201, 96.83665]
13.28416
[17.67178, 62.48418, 26.48314, 49.73674, 21.34884, 52.98266, 76.2009, 49.39388, 21.06437, 48.85645, 37.10908, 29.69851, 19.8193, 40.48159, 48.19345, 56.50562, 62.16133, 61.96548, 35.80282, 54.98403]
17.67178
[62.91572, 39.28823, 70.57809, 63.29954, 72.3295, 77.41518, 54.09028, 48.56796, 81.32521, 69.99786, 81.81473, 71.13796, 46.23478, 37.22824, 45.73219, 97.42905, 64.04475, 40.68526, 30.29565, 68.51214]
30.29565
[47.54054, 82.26642, 57.17504, 23.24321, 52.12539, 83.08763, 98.32166, 73.50838, 22.31056, 19.52236, 67.81934, 60.33738, 33.70979, 33.45961, 71.46854, 87.02598, 88

In [98]:
# 결과확인 : FC에 배정데이터 결합
final01 = pd.merge(em01, all01, how = 'inner', on = 'em_num')
final01

,em_num,em_name,money,latitude,hardness,cu_num,distance
0,10,K,99,126.867068,37.205859,11.0,1.60158
1,2,B,48,126.569316,37.122076,17.0,13.28416
2,1,A,41,126.861717,37.874623,10.0,20.36569
3,4,E,30,126.058275,37.535272,19.0,30.29565
4,5,F,21,126.775502,37.807801,10.0,19.52236
5,6,G,19,126.666107,37.427021,1.0,8.57989
6,7,H,17,126.244399,37.906478,14.0,25.55142
7,8,J,15,126.027184,37.185274,2.0,17.99214
8,9,I,15,126.224037,37.465374,19.0,18.34672
9,3,D,5,126.806583,37.531962,1.0,17.67178


In [73]:
cu01.loc[dis1.index(max(dis1))]

cu_num           20
cu_name           T
latitude     126.58
hardness    37.9929
Name: 19, dtype: object

In [75]:
cu01.loc[dis1.index(max(dis1))-1,"cu_num"]

19

In [60]:
cu01.loc[dis1.index(max(dis1))+1,"cu_num"]

4

In [13]:
dis1.index(max(dis1))

1

In [38]:
t1 = pd.DataFrame([1,2])
t2 = pd.DataFrame([1,2])
pd.concat([t1, t2])


,0
0,1
1,2
0,1
1,2


In [51]:
# df01 = pd.DataFrame({'em_num':[1], 'cu_num':[1]}, index=['2'])
# df01 = pd.DataFrame({'em_num':[1], 'cu_num':[1]})
df01 = pd.DataFrame({'em_num':[], 'cu_num':[]})
df01

,em_num,cu_num


In [8]:
# 서울시청 126.97843, 37.56668
# 강남역   127.02758, 37.49794
# 방법1)
GeoUtil.get_euclidean_distance(126.97843, 37.56668, 127.02758, 37.49794)





0.0845

In [9]:
# 방법2)
GeoUtil.get_harversion_distance(126.97843, 37.56668, 127.02758, 37.49794)


8.78676

In [ ]:


class GeoUtil:
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi/180)
    
    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        R = 6371 # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2-x1)    
        dLat = GeoUtil.degree2radius(y2-y1)

        a = math.sin(dLat/2) * math.sin(dLat/2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
              *math.cos(GeoUtil.degree2radius(y2)) \
              *math.sin(dLon/2) * math.sin(dLon/2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return round(R * b, round_decimal_digits)

    @staticmethod
    def get_euclidean_distance(x1, y1, x2, y2, round_decimal_digits=5):        
        """
        유클리안 Formula 이용하여 (x1,y1)과 (x2,y2) 점의 거리를 반환
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        dLon = abs(x2-x1) # 경도 차이
        if dLon >= 180:   # 반대편으로 갈 수 있는 경우
            dLon -= 360   # 반대편 각을 구한다
        dLat = y2-y1      # 위도 차이
        return round(math.sqrt(pow(dLon,2)+pow(dLat,2)),round_decimal_digits)

class GeoUtil:
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi/180)
    
    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        R = 6371 # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2-x1)    
        dLat = GeoUtil.degree2radius(y2-y1)

        a = math.sin(dLat/2) * math.sin(dLat/2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
              *math.cos(GeoUtil.degree2radius(y2)) \
              *math.sin(dLon/2) * math.sin(dLon/2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return round(R * b, round_decimal_digits)

    @staticmethod
    def get_euclidean_distance(x1, y1, x2, y2, round_decimal_digits=5):        
        """
        유클리안 Formula 이용하여 (x1,y1)과 (x2,y2) 점의 거리를 반환
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        dLon = abs(x2-x1) # 경도 차이
        if dLon >= 180:   # 반대편으로 갈 수 있는 경우
            dLon -= 360   # 반대편 각을 구한다
        dLat = y2-y1      # 위도 차이
        return round(math.sqrt(pow(dLon,2)+pow(dLat,2)),round_decimal_digits)